In [5]:
import os
import openai
import json

In [49]:
openai_api_key = os.getenv("OPENAI_API_KEY")
if openai == None:
    print("OPENAI_API_KEY environment variable not set")
    openai_api_key = input("Enter your API key: ")
openai.api_key = openai_api_key

In [51]:
model_list = openai.Model.list()
print([m["id"] for m in dict(model_list)["data"]])
standard_model = "text-davinci-003"

['whisper-1', 'babbage', 'davinci', 'text-davinci-edit-001', 'babbage-code-search-code', 'text-similarity-babbage-001', 'code-davinci-edit-001', 'text-davinci-001', 'ada', 'babbage-code-search-text', 'babbage-similarity', 'code-search-babbage-text-001', 'text-curie-001', 'gpt-3.5-turbo-16k-0613', 'code-search-babbage-code-001', 'text-ada-001', 'text-similarity-ada-001', 'curie-instruct-beta', 'gpt-3.5-turbo-0301', 'ada-code-search-code', 'ada-similarity', 'code-search-ada-text-001', 'text-search-ada-query-001', 'davinci-search-document', 'ada-code-search-text', 'text-search-ada-doc-001', 'davinci-instruct-beta', 'text-similarity-curie-001', 'code-search-ada-code-001', 'ada-search-query', 'text-search-davinci-query-001', 'curie-search-query', 'davinci-search-query', 'babbage-search-document', 'ada-search-document', 'text-search-curie-query-001', 'gpt-3.5-turbo', 'text-search-babbage-doc-001', 'gpt-3.5-turbo-0613', 'curie-search-document', 'text-search-curie-doc-001', 'babbage-search-que

In [58]:
response = openai.Completion.create(
    model=standard_model, 
    prompt="Question: \"Say OK if you understand.\"\nAnswer: \"", 
    max_tokens=3,
    stop=["\""])

In [59]:
print(response["choices"][0]["text"])

OK.


In [63]:
def ask_for_stackoverflow_answer(stackoverflow_question):
    response = openai.Completion.create(
        model=standard_model, 
        prompt="Question on StackOverflow: \"" + stackoverflow_question + 
                "\"\nGive a short and concise answer, including code snippets only if necessary. Answer in typical StackOverflow manner and the required markdown: \"",
        max_tokens=1000,
        stop=["\""])

    print(response["choices"][0]["text"])

In [64]:
ask_for_stackoverflow_answer("How do I get the current date in Python?")



The simplest way to get the current date is to use the [`datetime` module](https://docs.python.org/3/library/datetime.html) in Python's standard library:

```
import datetime

today = datetime.date.today()
print(today)
```

Note that you will get a datetime.date object, which holds the date but not the time.
